In [16]:
import numpy as np
import numpy.random as random
import pandas as pd
import scipy as sp
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

%precision 3


'%.3f'

In [17]:
import sklearn

__단순회귀분석__

---

회귀 분석은 주어진 데이터에서 변수 사이에 어떠한 관계를 가정하고 이 관계를 가장 잘 표현하는 수식을 구하는 것이다.

회귀 분석은 설명변수를 활용해 목표변수를 예측하며, 지도학습(Supervised Learning)의 한 종류이며 학습할 때 데이터에 정답을 하나씩 대응시켜 관계를 계산하는 기초적인 방법이다.

회귀분석은 결괏값(목표변수)과 입력값(설명변수)의 관계에서 입력 변수가 하나인 것과 둘 이상인 것으로 나눌 수 있다. 전자를 단순회귀분석, 후자를 다중회귀분석이라고 한다.

_선형회귀분석_

선형회귀분석은 입력값과 결괏값 사이에 선형관계($y = ax + b$)를 가정한다.

선형회귀분석은 사이킷런 라이브러리의 sklearn.linear_model을 이용해 간단하게 실행할 수 있다.

linear_model을 임포트한 후 인스턴스를 생성할 수 있다



In [18]:
from sklearn import linear_model
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score

cali = fetch_california_housing()

#선형회귀 인스턴스 생성
reg = linear_model.LinearRegression()
X = pd.DataFrame(cali.target)
Y = pd.DataFrame(cali.data, columns= ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude' ])
y = np.arange(0, 8)

# 8개의 설명변수의 단순선형회귀분석 진행
for col_num in y:
    reg.fit(X, Y.iloc[:, col_num])
    print('설명변수명 : {}'.format(Y.columns[col_num]))
    print('회귀계수:', reg.coef_)
    print('절편:', reg.intercept_)
    print('결정계수 : {:.3f}'.format(reg.score(X, Y.iloc[:, col_num])))
    print('')
    

설명변수명 : MedInc
회귀계수: [1.133]
절편: 1.5273748109684981
결정계수 : 0.473

설명변수명 : HouseAge
회귀계수: [1.152]
절편: 26.256557529056042
결정계수 : 0.011

설명변수명 : AveRooms
회귀계수: [0.326]
절편: 4.755085968678553
결정계수 : 0.023

설명변수명 : AveBedrms
회귀계수: [-0.019]
절편: 1.136348304483483
결정계수 : 0.002

설명변수명 : Population
회귀계수: [-24.191]
절편: 1475.5162923505427
결정계수 : 0.001

설명변수명 : AveOccup
회귀계수: [-0.214]
절편: 3.5125940332016126
결정계수 : 0.001

설명변수명 : Latitude
회귀계수: [-0.267]
절편: 36.18383160176508
결정계수 : 0.021

설명변수명 : Longitude
회귀계수: [-0.08]
절편: -119.40461582245466
결정계수 : 0.002



다음 코드는 설명변수($X$)와 목표변수($Y$) 데이터를 대상으로 fit 함수를 사용해 예측 모델을 구축한다.

fit 함수는 최소제곱법으로 회귀계수 $a$와 절편 $b$를 계산한다. 최소제곱법은 실제 목표변수 데이터 값과 예측한 목표변수 데이터 값의 차이를 제곱하고 모두 더한 값이 최소가 될 때의 계수와 절편을 구한다.

최소제곱법(least square method)은 산포도를 비교적 잘 설명할 수 있는 직선식의 추정에 그 목적이 있다.

직선의 식은 기울기와 $y$절편만 있으면 결정된다. $y$절편을 $\beta_0$, 기울기를 $\beta_1$이라 하고 최종 결정될 최소자승법에 의한 직선식은 다음과 같다.

$$
Y = \beta_0 + \beta_1 \cdot X
$$

여기서 $\beta_1$(기울기)은 $x$와 $y$의 표봉공분산을 $x$의 표본공분산의 제곱으로 나누어 계산한다.

$$
\beta_1 = \frac{s_{xy}}{s^2_r}
$$

$y$절편은 표본 $y$의 평균값에 표본 $x$의 평균값과 위에서 계산한 $\beta_1$을 곱한 값을 차감하여 구한다.

$$
\beta_0 = \bar{y} - \beta_1 \cdot \bar{x}
$$

최소제곱법을 통하여 추정한 회귀식이 다음과 같다고 할 때,

$$
\hat{y} = \hat{\beta_0} + \hat{\beta_1} \cdot x
$$

$\hat{y}$은 종속변수의 추정치에 해당한다. $\hat{\beta_0}$은 추정 회귀식의 $y$절편이고 $\hat{\beta_1}$은 기울기이다.

종속변수 $y$와 독립변수 $x$의 표본 순서쌍이 $n$개라 할 때 각 순서쌍을 다음과 같이 정의한다.

$$
(x_1, y_1), (x_2, y_2), \cdots, (x_i, y_i), \cdots, (x_n, y_n)
$$

추정된 식에 의하여 $x$와 $\hat{y}$의 순서쌍은 다음과 같다

$$
(x_1, \hat{y_1} = \hat{\beta_0} + \hat{\beta_1} \cdot x_1), (x_2, \hat{y_2} = \hat{\beta_0} + \hat{\beta_1} \cdot x_2), \cdots, (x_n, \hat{y_n} = \hat{\beta_0} + \hat{\beta_1} \cdot x_n)
$$

여기서 실체 관측된 $y_i$와 추정된 $\hat{y_i} = \hat{\beta_0} + \hat{\beta_1} \cdot x_i$의 차이를 잔차(residual), 즉 $e_i$라 한다. 즉, 각 관찰값에 대한 잔차는 $e_i = y_i - \hat{y_i}$이다. 위의 $n$개의 변수 쌍에서 해당 잔차의 합은 언제나 0이다. 이는 $y_i$의 평균과 $\hat{y_i}$이 같기 때문에 $e_i$의 평균은 0인 것이다. 직관적으로 양의 잔차가 음의 잔차에 의해서 상쇄된다고 보아도 무방하다.

$$
\sum_{i=1}^{n}e_i = \sum_{i=1}^{n}(y_i - \hat{y_i}) = 0
$$

최소제곱법은 잔차(residual) 제곱의 합이 최소가 되도록 하는 추세선을 계산하는 것이다. 잔차 제곱의 합을 $SSE$(Sum of Squared Error)라 한다.

$$
SSE = \sum_{i=1}^{n}e^2_i = \sum_{i=1}^{n}(y_i - \hat{y_i})^2 = \sum_{i=1}^{n}(y_i - \hat{\beta_0} - \hat{\beta_1} \cdot x_i)^2
$$

최소제곱법에 의하여 $SSE$가 최소가 되는 회귀식 $\hat{y} = \hat{\beta_0} + \hat{\beta_1} \cdot x$를 추정한다. 추정 모수는 $\hat{\beta_0}, \hat{\beta_1}$이다.

$\hat{\beta_1}$은 $x$와 $y$의 표본공분산을 $x$의 표본공분산의 제곱으로 나누어 계산한다.

$$
\hat{b_1} = \frac{s_{xy}}{s^2_x}
$$

$\hat{\beta_0}$은 위에서 계사한 $\hat{\beta_1}$ 값과 종속변수 $y$의 표본평균과 독립변수 $x$의 표본평균을 이용하여 다음과 같이 산출한다.

$$
\hat{\beta_0} = \bar{y} - \hat{\beta_1} \cdot \bar{x}
$$

이렇게 계산된 $\hat{\beta_0}, \hat{\beta_1}$ 이 $SSE$를 최소가 되도록 하는 추정인 것이다.


추가로 해당 추세선이 실제 산포도를 얼마나 정확히 추정하고 있는지 또한 계산할 수 있다.

$\bar{y}$를 $y_1$부터 $y_n$까지 값의 평균이라 정의하자.

$$
\bar{y} = \frac{1}{n} \sum_{i=1}^{n} y_i
$$

$y$변수가 평균 대비 얼마나 변동이 있는지를 나타내는 총변동은 $SST$(Sum off Squared Total)는 다음과 같이 계산한다.

$$
SST = \sum_{i_1}^{n}(y_i - \bar{y})^2
$$

회귀식에 의하여 추정된 종속변수 데이터 $\hat{y_i}$과 표본평균 $\bar{y}$의 차이에 대한 제곱합을 $SSR$(Sum of Squared Regression)이라 정의하고 해당 수식은 다음과 같이 나타낸다.

$$
SSR = \sum_{i=i}^{n}(\hat{y_i} - \bar{y})^2
$$

총변동 $SST$는 $SSR$과 $SSE$의 합과 같다.

$$
SST = SSR + SSE
\\
\sum_{i=1}^{n}(y_i - \bar{y})^2 = \sum_{i=1}^{n}(\hat{y_i} - \bar{y})^2 + \sum_{i=1}^{n}(y_i - \hat{y_i})^2
$$

회귀식의 정확성은 결정계수의 크기로 판단할 수 있다.

$SSR$은 회귀식으로 설명되는 편차제곱합이고, $SSE$는 회귀식으로 설명되지 않는 편차제곱합이다. 전체 편차제곱합인 $SST$가 있는데, 이 중 회귀식으로 설명되는 편차제곱합 $SSR$이 높으면 높을수록 더 정확한 회귀식이다.

$$
R^2 = \frac{SSR}{SST}
$$

<p>&nbsp;</p>

$$
SST = SSR + SSE
\\
\Longleftrightarrow 1 = \frac{SSR}{SST} + \frac{SSE}{SST}
\\
\Longleftrightarrow \frac{SSR}{SST} = 1 - \frac{SSE}{SST}
$$

이므로, 다음과 같이 결정계수를 표현할 수 있다.

$$
R^2 = 1 - \frac{SSE}{SST} = \frac{SSR}{SST}
$$

$SSR$은 $SST$보다 언제나 작거나 같으므로 결정계수 $R^2$은 0과 1 사이의 값을 갖는다.

$$
0 \leq R^2 \leq 1
$$

만약 $R^2$이 65%라면 추정 회귀식의 설명력이 65%라는 의미이다.


---

<p>&nbsp;</p>

__다중회귀분석__

---

설명변수가 하나인 단순회귀의 원리를 확장해 설명변수가 여러 개일 때, 다중회귀이다. 회귀계수는 회귀모형이 예측한 값과 데이터의 실젯값(목표변수)과의 차이를 제곱하고 모두 더한 값이 최소가 되도록 만드는 값으로 추정한다.

목표변수가 $y$, 설명변수가 $x_1, x_2$인 다중회귀분석은 선형관계를 가정할 때 다음과 같은 모형을 이룬다.

$$
y = \beta_0 + \beta_1 \cdot x_1 + \beta_2 \cdot x_2 + \varepsilon
$$

위의 식이 모집단의 다중회귀분석을 위한 모형이다. 위 식에서는 $y$는 종속변수, $x_1, x_2$는 설명변수, $\beta_0$는 $y$절편, $\beta_1$과 $\beta_2$는 각각 $x_1, x_2$에 대한 회귀계수, $\varepsilon$은 오차항을 의미한다. 여기서 오차항 $\varepsilon$은 관찰할 수 없는 요소이고, $\beta_0, \beta_1, \beta_2$가 추정하고자 하는 모수이다.


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

X = cali.target
y = cali.data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.5, random_state=0)

full_model = sm.OLS(X_train, y_train)
fitted_full_model = full_model.fit()
fitted_full_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.895
Model:                            OLS   Adj. R-squared (uncentered):              0.895
Method:                 Least Squares   F-statistic:                          1.102e+04
Date:                Thu, 08 Jun 2023   Prob (F-statistic):                        0.00
Time:                        10:32:59   Log-Likelihood:                         -11903.
No. Observations:               10320   AIC:                                  2.382e+04
Df Residuals:                   10312   BIC:                                  2.388e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.5282      0.006     87.371      0.000       0.516       0.540
x2             0.0153      0.001     23.780      0.000       0.014       0.017
x3            -0.2042      0.009    -23.175      0.000      -0.221      -0.187
x4             0.9064      0.041     22.052      0.000       0.826       0.987
x5          6.818e-06   6.95e-06      0.981      0.326    -6.8e-06    2.04e-05
x6            -0.0089      0.001     -7.556      0.000      -0.011      -0.007
x7            -0.0612      0.005    -12.217      0.000      -0.071      -0.051
x8            -0.0159      0.002    -10.027      0.000      -0.019      -0.013
==============================================================================
Omnibus:                     2113.064   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5649.893
Skew:                           1.101   Prob(JB):                         0.00
Kurtosis:                       5.879   Cond. No.                     1.03e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.03e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

cali_data_frame = pd.DataFrame(cali.data, columns= ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude' ])

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(cali_data_frame.values, i) for i in range(cali_data_frame.shape[1])]
vif['features'] = cali_data_frame.columns
vif

,VIF Factor,features
0,11.511140,MedInc
1,7.195917,HouseAge
2,45.993601,AveRooms
3,43.590314,AveBedrms
4,2.935745,Population
5,1.095243,AveOccup
6,559.874071,Latitude
7,633.711654,Longitude



_모델 구축 및 모델 평가 과정_

> **1.** 모델 구축을 위한 클래스의 인스턴스 생성
>> model = LinearRegression()

> **2.** 데이터를 설명변수와 목표변수로 나눔
>> $X$와 $y$

> **3.** 훈련 데이터와 테스트 데이터로 분할
>> train_test_spilit(X, y, test_size = 0.5, random_state = 0)

> **4.** 훈련 데이터를 이용하여 학습
>> model.fit(X_train, y_train)

> **5.** 모델 성능을 테스트 데이터로 평가
>> model.score(X_test, y_test)

In [ ]:
import requests, zipfile
import io

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

auto.columns = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
                'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
                'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
                'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

In [ ]:
print('자동차 데이터 형태:{}'.format(auto.shape))

자동차 데이터 형태:(205, 26)


In [ ]:
auto.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [ ]:
#물음표 개수 확인

auto = auto[['price', 'horsepower','width','height']]
auto.isin(['?']).sum()

price         4
horsepower    2
width         0
height        0
dtype: int64

In [ ]:
#NaN으로 변환 후 제거

auto = auto.replace('?', np.nan).dropna()
print('자동차 데이터 형태:{}'.format(auto.shape))

자동차 데이터 형태:(199, 4)


In [ ]:
print('데이터 형 확인(형 변환 전)\n{}\n'.format(auto.dtypes))

데이터 형 확인(형 변환 전)
price          object
horsepower     object
width         float64
height        float64
dtype: object



In [ ]:
auto = auto.assign(price=pd.to_numeric(auto.price))
auto = auto.assign(horsepower=pd.to_numeric(auto.horsepower))
print('데이터 형 확인(형 변환 후)\n{}'.format(auto.dtypes))

데이터 형 확인(형 변환 후)
price           int64
horsepower      int64
width         float64
height        float64
dtype: object


In [ ]:
auto.corr()

#상관관계가 높은 변수들을 다중회귀의 모델에 함께 설명변수로 투입하면 다중공선성(multicollinearity)이 발생할 가능성이 있음
#변수 간의 상관관계가 높아 회귀계수의 분산이 커지고, 결과적으로 회귀계수의 통계적 유의성이 없어짐
#이러한 이유로 상관관계가 높은 변수 그룹을 대표하는 변수만을 다중회귀모델에 투입함

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


In [ ]:
#데이터 분할(훈련 데이터와 테스트 데이터)을 위한 임포트
from sklearn.model_selection import train_test_split

#다중회귀 모델 구축을 위한 임포트
from sklearn.linear_model import LinearRegression

#목표변수로 price를 지정, 설명변수로는 price 외의 다른 변수를 지정
X = auto.drop('price', axis=1)
y = auto['price']

#훈련 데이터와 테스트 테이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

#다중회귀 클래스 초기화와 학습
model = LinearRegression()
model.fit(X_train, y_train)

#결정계수 출력
print('결정계수(train):{:.3f}'.format(model.score(X_train, y_train)))
print('결정계수(test):{:.3f}'.format(model.score(X_test, y_test)))

print('\n회귀계수\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('절편: {:.3f}'.format(model.intercept_))

결정계수(train):0.733
결정계수(test):0.737

회귀계수
horsepower      81.651078
width         1829.174506
height         229.510077
dtype: float64
절편: -128409.046
